In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("../dataset/SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.ensemble import GradientBoostingClassifier

## Build our own Grid-search

In [7]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [8]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(X_test)
    precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision, 3), round(recall, 3), 
        round((y_pred==y_test).sum()/len(y_pred), 3)))

In [ ]:
for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.022 / Accuracy: 0.879
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.932 / Recall: 0.696 / Accuracy: 0.956
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.857 / Recall: 0.739 / Accuracy: 0.952
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.014 / Accuracy: 0.878
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.949 / Recall: 0.812 / Accuracy: 0.971
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.871 / Recall: 0.833 / Accuracy: 0.964
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.877
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.908 / Recall: 0.855 / Accuracy: 0.971
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.87 / Recall: 0.826 / Accuracy: 0.963
Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.014 / Accuracy: 0.878
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.899 / Recall: 0.841 / Accuracy: 0.969
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.876 / Recall: 0.819 / Acc